In [ ]:
# Load libraries
install.packages("fastDummies")
install.packages("ggplot2")
library(fastDummies)
library(tidyr)
library(dplyr)
library(ggplot2)

# Read in data
all <- read.csv('profile_all_cleaned.csv')
subset <- read.csv('profile_subset_cleaned.csv')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
head(all)
nrow(all)
nrow(subset)

,CUSTOMER_NUMBER,PRIMARY_GROUP_NUMBER,FREQUENT_ORDER_TYPE,FIRST_DELIVERY_DATE,ON_BOARDING_DATE,COLD_DRINK_CHANNEL,TRADE_CHANNEL,SUB_TRADE_CHANNEL,LOCAL_MARKET_PARTNER,CO2_CUSTOMER,ZIP_CODE,total_gal_2023,total_gal_2024,avg_gal_per_year,cases_total,fountain_total,threshold_400_gal,percent_change,fountain_only
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<lgl>
1,500245678,NA,SALES REP,2018-03-19,2015-03-11,EVENT,OUTDOOR ACTIVITIES,OTHER OUTDOOR ACTIVITIES,TRUE,TRUE,66508,370.0,383.5,376.75,361.0,392.5,FALSE,3.648649,FALSE
2,500245685,NA,OTHER,2018-03-02,2015-08-18,DINING,FAST CASUAL DINING,PIZZA FAST FOOD,TRUE,TRUE,21913,601.5,482.0,541.75,61.0,1022.5,TRUE,-19.866999,FALSE
3,500245686,8333,CALL CENTER,2023-03-07,2015-08-05,GOODS,SPECIALIZED GOODS,OTHER GOODS,TRUE,FALSE,1350,17.5,18.5,18.00,36.0,0.0,FALSE,5.714286,FALSE
4,500245687,NA,OTHER,2018-03-19,2015-08-06,EVENT,OUTDOOR ACTIVITIES,OTHER OUTDOOR ACTIVITIES,TRUE,TRUE,42252,125.0,147.5,136.25,0.0,272.5,FALSE,18.000000,TRUE
5,500245689,NA,OTHER,2018-02-28,2015-08-25,DINING,LICENSED HOSPITALITY,OTHER LICENSED HOSPITALITY,TRUE,FALSE,42031,546.5,621.0,583.75,287.5,880.0,TRUE,13.632205,FALSE
6,500245690,2272,SALES REP,2018-03-01,2015-08-25,DINING,FAST CASUAL DINING,OTHER FAST FOOD,TRUE,TRUE,1245,325.0,300.5,312.75,123.0,502.5,FALSE,-7.538462,FALSE


[1] 30478

[1] 1324

In [ ]:
## SUBSET

# 1. Customers under 400-gallon threshold
under_threshold <- subset(subset, threshold_400_gal == FALSE)

# 2. YoY growth for under-threshold customers
under_threshold$yoy_growth <- (under_threshold$total_gal_2024 - under_threshold$total_gal_2023) / under_threshold$total_gal_2023

# 3. Number of such customers
n_sub400 <- nrow(under_threshold)

# 4. Gallons lost in 2024 if we drop them
gallons_lost_2024 <- sum(under_threshold$total_gal_2024, na.rm = TRUE)

# 5. Total 2024 gallons (all customers in subset)
total_gal_2024_all <- sum(subset$total_gal_2024, na.rm = TRUE)

# 6. Percent lost
percent_lost <- gallons_lost_2024 / total_gal_2024_all * 100

# 7. Summary
summary_df <- data.frame(
  sub400_customer_count = n_sub400,
  gallons_lost_if_dropped = gallons_lost_2024,
  total_gallons_2024 = total_gal_2024_all,
  percent_of_total_lost = percent_lost
)

print(summary_df)



  sub400_customer_count gallons_lost_if_dropped total_gallons_2024
1                  1141                122426.5           292526.5
  percent_of_total_lost
1              41.85141


In [ ]:
## ALL

# 1. Customers under 400-gallon threshold
under_threshold <- subset(all, threshold_400_gal == FALSE)

# 2. YoY growth for under-threshold customers
under_threshold$yoy_growth <- (under_threshold$total_gal_2024 - under_threshold$total_gal_2023) / under_threshold$total_gal_2023

# 3. Number of such customers
n_sub400 <- nrow(under_threshold)

# 4. Gallons lost in 2024 if we drop them
gallons_lost_2024 <- sum(under_threshold$total_gal_2024, na.rm = TRUE)

# 5. Total 2024 gallons (all customers)
total_gal_2024_all <- sum(all$total_gal_2024, na.rm = TRUE)

# 6. Percent lost
percent_lost <- gallons_lost_2024 / total_gal_2024_all * 100

# 7. Summary
summary_df_ALL <- data.frame(
  sub400_customer_count_ALL = n_sub400,
  gallons_lost_if_dropped_ALL = gallons_lost_2024,
  total_gallons_2024_ALL = total_gal_2024_all,
  percent_of_total_lost_ALL = percent_lost
)

print(summary_df_ALL)



  sub400_customer_count_ALL gallons_lost_if_dropped_ALL total_gallons_2024_ALL
1                     22772                     2928766               19691874
  percent_of_total_lost_ALL
1                  14.87297


In [ ]:
drop_by_percent <- subset[subset$percent_change < 0, ]
count_percent <- nrow(drop_by_percent)
gallons_lost <- sum(drop_by_percent$total_gal_2024, na.rm = TRUE)
percent_lost <- gallons_lost_percent / total_gal_2024_all * 100

summary_df <- data.frame(
  count_percent,
  gallons_lost_percent,
  percent_lost_percent
)

print(summary_df)

  count_percent gallons_lost_percent percent_lost_percent
1           597               131919            0.6699159


In [ ]:
# Filter: Customers under 400-gallon threshold
under_threshold <- subset(subset, threshold_400_gal == FALSE)

# Split into two groups based on percent_change
growing <- under_threshold[under_threshold$percent_change >= 0, ]
shrinking <- under_threshold[under_threshold$percent_change < 0, ]

# Count of customers
n_growing <- nrow(growing)
n_shrinking <- nrow(shrinking)

# Gallons lost in 2024 if either group were dropped
gallons_lost_growing <- sum(growing$total_gal_2024, na.rm = TRUE)
gallons_lost_shrinking <- sum(shrinking$total_gal_2024, na.rm = TRUE)

# Total 2024 gallons (for comparison)
total_gal_2024_all <- sum(subset$total_gal_2024, na.rm = TRUE)

# Percent of total lost if either group dropped
percent_lost_growing <- gallons_lost_growing / total_gal_2024_all * 100
percent_lost_shrinking <- gallons_lost_shrinking / total_gal_2024_all * 100

# Summary table
summary_df <- data.frame(
  group = c("Under 400 & Growing (>= 0%)", "Under 400 & Shrinking (< 0%)"),
  customer_count = c(n_growing, n_shrinking),
  gallons_lost_if_dropped = c(gallons_lost_growing, gallons_lost_shrinking),
  percent_of_total_2024_lost = c(percent_lost_growing, percent_lost_shrinking)
)

print(summary_df)


                         group customer_count gallons_lost_if_dropped
1  Under 400 & Growing (>= 0%)            648                72657.46
2 Under 400 & Shrinking (< 0%)            493                49769.00
  percent_of_total_2024_lost
1                   24.83791
2                   17.01350
